In [1]:
!pip install torch torchvision torchaudio
!pip install ogb
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 24.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

In [ ]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

wandb.login(key="Your-api-key")
Dataset_name = "NCI1"

# *Baseline*

In [3]:
import os
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)
    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.lin(x)
        return F.log_softmax(x, dim=1)

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)

def run_seed(seed):
    set_seed(seed)
    
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name)

    # Filter out None graphs and create x if missing
    clean_dataset = []
    for data in dataset:
        if data is None:
            continue
        if data.x is None:
            data.x = torch.ones((data.num_nodes, 1), dtype=torch.float)
        clean_dataset.append(data)

    if len(clean_dataset) == 0:
        raise RuntimeError(f"No valid graphs in dataset. Check your {Dataset_name} installation.")

    dataset = clean_dataset

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.shape[1]
    out_channels = max([data.y.max().item() for data in dataset]) + 1
    model = GCN(in_channels, 64, out_channels).to(device)

    # Shuffle using random.shuffle instead of dataset.shuffle()
    random.shuffle(dataset)

    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_baseline_seed_{seed}",
        group=f"GCN_{Dataset_name}_baseline",
        job_type="baseline",
        reinit=True,
        config={
            "model":"GCN",
            "dataset":Dataset_name,
            "hidden_channels":64,
            "lr":0.001,
            "epochs":EPOCHS,
            "task":"Baseline",
            "seed":seed
        }
    )

    def train():
        model.train()
        total_loss = 0.0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index, batch.batch)
            loss = F.nll_loss(out, batch.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(train_loader)

    def test(loader):
        model.eval()
        correct = 0
        for batch in loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index, batch.batch)
            pred = out.argmax(dim=1)
            correct += (pred == batch.y).sum().item()
        return correct / len(loader.dataset)

    for epoch in range(1, EPOCHS + 1):
        loss = train()
        test_acc = test(test_loader)
        train_acc = test(train_loader)
        
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})

    wandb.finish()

if __name__ == "__main__":
    for s in range(1, 31):
        run_seed(s)


Processing...
Done!
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Seed 1 | Epoch 001 | Loss: 0.6778 | Test Acc: 0.5964
Seed 1 | Epoch 002 | Loss: 0.6727 | Test Acc: 0.5964
Seed 1 | Epoch 003 | Loss: 0.6664 | Test Acc: 0.6413
Seed 1 | Epoch 004 | Loss: 0.6628 | Test Acc: 0.6457
Seed 1 | Epoch 005 | Loss: 0.6576 | Test Acc: 0.6323
Seed 1 | Epoch 006 | Loss: 0.6534 | Test Acc: 0.6323
Seed 1 | Epoch 007 | Loss: 0.6492 | Test Acc: 0.6457
Seed 1 | Epoch 008 | Loss: 0.6454 | Test Acc: 0.6413
Seed 1 | Epoch 009 | Loss: 0.6410 | Test Acc: 0.6457
Seed 1 | Epoch 010 | Loss: 0.6356 | Test Acc: 0.6637
Seed 1 | Epoch 011 | Loss: 0.6314 | Test Acc: 0.6816
Seed 1 | Epoch 012 | Loss: 0.6292 | Test Acc: 0.6726
Seed 1 | Epoch 013 | Loss: 0.6338 | Test Acc: 0.6323
Seed 1 | Epoch 014 | Loss: 0.6283 | Test Acc: 0.6682
Seed 1 | Epoch 015 | Loss: 0.6253 | Test Acc: 0.6861
Seed 1 | Epoch 016 | Loss: 0.6259 | Test Acc: 0.6726
Seed 1 | Epoch 017 | Loss: 0.6256 | Test Acc: 0.6951
Seed 1 | Epoch 018 | Loss: 0.6234 | Test Acc: 0.6816
Seed 1 | Epoch 019 | Loss: 0.6213 | Test Acc: 

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▆▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▁▁▁▂▁▁▁▁▁▁▁
test_acc,▁▁▄▄▃▄▄▄▆▇▃▆▇▆██▆▇▆▇▇▇▆▄▅██▆█▇▇█▇▅▇▇█▆▆▇
train_acc,▁▁▃▃▄▃▃▄▇▆▇▇▇▇▇▇▇█▆▇▆▆▇█▇▇▇█▇█▇█▇▇▇██▇▇█
epoch,50
loss,0.61391
test_acc,0.6861
train_acc,0.71461


Seed 2 | Epoch 001 | Loss: 0.6795 | Test Acc: 0.5695
Seed 2 | Epoch 002 | Loss: 0.6708 | Test Acc: 0.5695
Seed 2 | Epoch 003 | Loss: 0.6672 | Test Acc: 0.5695
Seed 2 | Epoch 004 | Loss: 0.6606 | Test Acc: 0.5785
Seed 2 | Epoch 005 | Loss: 0.6538 | Test Acc: 0.5919
Seed 2 | Epoch 006 | Loss: 0.6458 | Test Acc: 0.6054
Seed 2 | Epoch 007 | Loss: 0.6418 | Test Acc: 0.6054
Seed 2 | Epoch 008 | Loss: 0.6337 | Test Acc: 0.6323
Seed 2 | Epoch 009 | Loss: 0.6287 | Test Acc: 0.6413
Seed 2 | Epoch 010 | Loss: 0.6279 | Test Acc: 0.6457
Seed 2 | Epoch 011 | Loss: 0.6243 | Test Acc: 0.6413
Seed 2 | Epoch 012 | Loss: 0.6218 | Test Acc: 0.6502
Seed 2 | Epoch 013 | Loss: 0.6190 | Test Acc: 0.6502
Seed 2 | Epoch 014 | Loss: 0.6175 | Test Acc: 0.6323
Seed 2 | Epoch 015 | Loss: 0.6156 | Test Acc: 0.6323
Seed 2 | Epoch 016 | Loss: 0.6175 | Test Acc: 0.6323
Seed 2 | Epoch 017 | Loss: 0.6135 | Test Acc: 0.6368
Seed 2 | Epoch 018 | Loss: 0.6135 | Test Acc: 0.6368
Seed 2 | Epoch 019 | Loss: 0.6103 | Test Acc: 

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇███
loss,█▇▇▆▆▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁
test_acc,▁▁▁▂▃▄▆▇█▇█▆▆▇▇▇▆▇▇▇▇▆▆▇▆▇▆▇▇▆█▆▆▆▆▆██▇▆
train_acc,▁▁▁▂▃▄▇▇▆▇▇▇▇▇▇██▇█▇▇▇▇▇▇▇▇▇██▇▆▇▇▇█▆██▇
epoch,50
loss,0.60373
test_acc,0.6278
train_acc,0.71011


...

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▆▆▄▄▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▂▂▂▁▁▁▁▁▂▂▁▁
test_acc,▁▂▂▂▁▁▃▂▁▆▆▇▆▇█▇▇▃▇▆█▇▆▇▇▇▇▇█▇▇▆██▇▇▇▇▇▇
train_acc,▁▂▃▄▄▄▄▄▄▇▆▇▇▇▇▇▅▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▆▇█▇▇▇▇
epoch,50
loss,0.6088
test_acc,0.69058
train_acc,0.70337


Seed 29 | Epoch 001 | Loss: 0.6779 | Test Acc: 0.6323
Seed 29 | Epoch 002 | Loss: 0.6723 | Test Acc: 0.6368
Seed 29 | Epoch 003 | Loss: 0.6705 | Test Acc: 0.6457
Seed 29 | Epoch 004 | Loss: 0.6648 | Test Acc: 0.6413
Seed 29 | Epoch 005 | Loss: 0.6620 | Test Acc: 0.6502
Seed 29 | Epoch 006 | Loss: 0.6638 | Test Acc: 0.6413
Seed 29 | Epoch 007 | Loss: 0.6555 | Test Acc: 0.6368
Seed 29 | Epoch 008 | Loss: 0.6494 | Test Acc: 0.6771
Seed 29 | Epoch 009 | Loss: 0.6452 | Test Acc: 0.6996
Seed 29 | Epoch 010 | Loss: 0.6440 | Test Acc: 0.7399
Seed 29 | Epoch 011 | Loss: 0.6415 | Test Acc: 0.7309
Seed 29 | Epoch 012 | Loss: 0.6373 | Test Acc: 0.6951
Seed 29 | Epoch 013 | Loss: 0.6391 | Test Acc: 0.6861
Seed 29 | Epoch 014 | Loss: 0.6345 | Test Acc: 0.7130
Seed 29 | Epoch 015 | Loss: 0.6336 | Test Acc: 0.7309
Seed 29 | Epoch 016 | Loss: 0.6302 | Test Acc: 0.7130
Seed 29 | Epoch 017 | Loss: 0.6313 | Test Acc: 0.7130
Seed 29 | Epoch 018 | Loss: 0.6302 | Test Acc: 0.7265
Seed 29 | Epoch 019 | Loss: 

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▂▁▁▁▁▁▁
test_acc,▁▁▂▂▂▁▄▅█▇▅▆▇▆▆▇▇▇▇▇▇▇▇▇▇▆█▆▇▆▇▅▇█▆█▆▇▇▇
train_acc,▁▂▃▃▄▄▅▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇███▇▆▇▇▇▇█▇█
epoch,50
loss,0.62018
test_acc,0.72197
train_acc,0.69213


Seed 30 | Epoch 001 | Loss: 0.6803 | Test Acc: 0.5964
Seed 30 | Epoch 002 | Loss: 0.6709 | Test Acc: 0.5964
Seed 30 | Epoch 003 | Loss: 0.6667 | Test Acc: 0.6009
Seed 30 | Epoch 004 | Loss: 0.6642 | Test Acc: 0.6054
Seed 30 | Epoch 005 | Loss: 0.6605 | Test Acc: 0.5964
Seed 30 | Epoch 006 | Loss: 0.6548 | Test Acc: 0.6009
Seed 30 | Epoch 007 | Loss: 0.6508 | Test Acc: 0.5964
Seed 30 | Epoch 008 | Loss: 0.6459 | Test Acc: 0.5874
Seed 30 | Epoch 009 | Loss: 0.6438 | Test Acc: 0.5919
Seed 30 | Epoch 010 | Loss: 0.6383 | Test Acc: 0.5964
Seed 30 | Epoch 011 | Loss: 0.6368 | Test Acc: 0.6009
Seed 30 | Epoch 012 | Loss: 0.6321 | Test Acc: 0.6368
Seed 30 | Epoch 013 | Loss: 0.6298 | Test Acc: 0.6637
Seed 30 | Epoch 014 | Loss: 0.6252 | Test Acc: 0.6278
Seed 30 | Epoch 015 | Loss: 0.6258 | Test Acc: 0.6278
Seed 30 | Epoch 016 | Loss: 0.6244 | Test Acc: 0.5964
Seed 30 | Epoch 017 | Loss: 0.6280 | Test Acc: 0.6323
Seed 30 | Epoch 018 | Loss: 0.6222 | Test Acc: 0.6413
Seed 30 | Epoch 019 | Loss: 

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,█▇▇▆▆▅▅▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▂▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁
test_acc,▂▂▂▂▂▂▁▁▂▂▄▄▂▅▅▅▇▅▆▆▇▇▆▆▇▅▇█▆█▆▇▇▆█▇█▅▅█
train_acc,▁▁▅▃▅▄▅▄▅▇█▇▄█▆█▇▇▇▇▇▇█▇▇▆▇█▇▇▇▆█▇▇▆▇▆█▇
epoch,50
loss,0.61082
test_acc,0.67265
train_acc,0.71011


# *BFS*

In [6]:
import os, random, numpy as np, torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import to_networkx
import networkx as nx
import wandb

EPOCHS = int(os.getenv("EPOCHS", "50"))
NUM_SEEDS = int(os.getenv("NUM_SEEDS", "30"))
PROJECT = f"GCN Graph classification({Dataset_name})_Gal"
ENTITY = "noynetanel1-tel-aviv-university"

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return F.log_softmax(self.lin(x), dim=1)

def set_seed(s):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    torch.cuda.manual_seed_all(s)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def add_bfs_rank(d):
    d = d.clone()
    if d.x is None:
        d.x = torch.ones((d.num_nodes, 1), dtype=torch.float)
    G = to_networkx(d, to_undirected=True)
    root = int(d.edge_index[0][0].item()) if d.edge_index.numel() > 0 else 0
    bfs_order = list(nx.bfs_tree(G, root)) if G.number_of_nodes() > 0 else []
    bfs_rank = torch.full((d.num_nodes, 1), -1.0, dtype=torch.float)
    L = max(1, len(bfs_order))
    for i, node in enumerate(bfs_order):
        bfs_rank[node] = float(i) / float(L)
    d.x = torch.cat([d.x, bfs_rank], dim=1)
    return d

def run_seed(seed):
    set_seed(seed)
    dataset = TUDataset(root='data/TUDataset', name=Dataset_name, transform=add_bfs_rank)
    torch.manual_seed(seed)
    dataset = dataset.shuffle()
    num_graphs = len(dataset)
    train_size = int(0.8 * num_graphs)
    train_dataset = dataset[:train_size]
    test_dataset = dataset[train_size:]
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    in_channels = dataset[0].x.size(1)
    model = GCN(in_channels, 64, dataset.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    wandb.init(
        project=PROJECT,
        entity=ENTITY,
        name=f"GCN_BFS_seed_{seed}",
        group=f"GCN_{Dataset_name}_BFS",
        job_type="bfs",
        reinit=True,
        config={"model":"GCN","hidden_channels":64,"epochs":EPOCHS,"lr":0.001,"dataset":"REDDIT-BINARY","task":"BFS","positional_encoding":"BFS","seed":seed}
    )

    def train():
        model.train()
        total_loss = 0.0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.batch)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(train_loader)

    def test(loader):
        model.eval()
        correct = 0
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            pred = out.argmax(dim=1)
            correct += int((pred == data.y).sum())
        return correct / len(loader.dataset)

    for epoch in range(1, EPOCHS + 1):
        loss = train()
        test_acc = test(test_loader)
        train_acc = test(train_loader)
        print(f"Seed {seed} | Epoch {epoch:03d} | Loss: {loss:.4f} | Test Acc: {test_acc:.4f}")
        wandb.log({"epoch": epoch, "loss": loss, "train_acc": train_acc, "test_acc": test_acc})
    wandb.finish()

if __name__ == "__main__":
    wandb.login()
    for s in range(1, NUM_SEEDS + 1):
        run_seed(s)


Seed 1 | Epoch 001 | Loss: 0.6811 | Test Acc: 0.5785
Seed 1 | Epoch 002 | Loss: 0.6696 | Test Acc: 0.5785
Seed 1 | Epoch 003 | Loss: 0.6660 | Test Acc: 0.5785
Seed 1 | Epoch 004 | Loss: 0.6612 | Test Acc: 0.6009
Seed 1 | Epoch 005 | Loss: 0.6558 | Test Acc: 0.6188
Seed 1 | Epoch 006 | Loss: 0.6480 | Test Acc: 0.6188
Seed 1 | Epoch 007 | Loss: 0.6400 | Test Acc: 0.6233
Seed 1 | Epoch 008 | Loss: 0.6318 | Test Acc: 0.6323
Seed 1 | Epoch 009 | Loss: 0.6222 | Test Acc: 0.6413
Seed 1 | Epoch 010 | Loss: 0.6232 | Test Acc: 0.6547
Seed 1 | Epoch 011 | Loss: 0.6143 | Test Acc: 0.6771
Seed 1 | Epoch 012 | Loss: 0.6022 | Test Acc: 0.6771
Seed 1 | Epoch 013 | Loss: 0.5930 | Test Acc: 0.6816
Seed 1 | Epoch 014 | Loss: 0.5871 | Test Acc: 0.6726
Seed 1 | Epoch 015 | Loss: 0.5849 | Test Acc: 0.6637
Seed 1 | Epoch 016 | Loss: 0.5754 | Test Acc: 0.6726
Seed 1 | Epoch 017 | Loss: 0.5695 | Test Acc: 0.6951
Seed 1 | Epoch 018 | Loss: 0.5617 | Test Acc: 0.6951
Seed 1 | Epoch 019 | Loss: 0.5601 | Test Acc: 

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▆▆▆▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▂▂▁
test_acc,▁▁▁▂▃▃▃▄▄▅▅▅▅▆▆▆▆▆▇▆▇▆▇█▇▇▇██▇█▇█████▇▇█
train_acc,▁▁▁▁▂▃▅▅▅▅▆▅▆▆▆▇▆▆▇▇▇▇▇█▇████▇█▇██████▇█
epoch,50
loss,0.49746
test_acc,0.73543
train_acc,0.76854


Seed 2 | Epoch 001 | Loss: 0.6774 | Test Acc: 0.6188
Seed 2 | Epoch 002 | Loss: 0.6744 | Test Acc: 0.6188
Seed 2 | Epoch 003 | Loss: 0.6693 | Test Acc: 0.6188
Seed 2 | Epoch 004 | Loss: 0.6646 | Test Acc: 0.6906
Seed 2 | Epoch 005 | Loss: 0.6595 | Test Acc: 0.6861
Seed 2 | Epoch 006 | Loss: 0.6504 | Test Acc: 0.6726
Seed 2 | Epoch 007 | Loss: 0.6456 | Test Acc: 0.7265
Seed 2 | Epoch 008 | Loss: 0.6355 | Test Acc: 0.7040
Seed 2 | Epoch 009 | Loss: 0.6264 | Test Acc: 0.7309
Seed 2 | Epoch 010 | Loss: 0.6188 | Test Acc: 0.7220
Seed 2 | Epoch 011 | Loss: 0.6116 | Test Acc: 0.7220
Seed 2 | Epoch 012 | Loss: 0.6138 | Test Acc: 0.6996
Seed 2 | Epoch 013 | Loss: 0.6015 | Test Acc: 0.7265
Seed 2 | Epoch 014 | Loss: 0.5938 | Test Acc: 0.7265
Seed 2 | Epoch 015 | Loss: 0.5979 | Test Acc: 0.7399
Seed 2 | Epoch 016 | Loss: 0.5871 | Test Acc: 0.7354
Seed 2 | Epoch 017 | Loss: 0.5784 | Test Acc: 0.7309
Seed 2 | Epoch 018 | Loss: 0.5764 | Test Acc: 0.7265
Seed 2 | Epoch 019 | Loss: 0.5712 | Test Acc: 

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁
test_acc,▁▁▁▄▄▆▅▆▆▆▆▆▆▆▆▆▇▇█▇█▇▆████▇▇█▇██▇███▇▇▇
train_acc,▁▁▁▂▃▄▄▅▆▅▆▆▆▆▆▆▇▇▇▇▇▇▆▇███▇██▇▇██████▇█
epoch,50
loss,0.50919
test_acc,0.75785
train_acc,0.76517


Seed 3 | Epoch 001 | Loss: 0.6737 | Test Acc: 0.5426
Seed 3 | Epoch 002 | Loss: 0.6679 | Test Acc: 0.5426
Seed 3 | Epoch 003 | Loss: 0.6630 | Test Acc: 0.5426
Seed 3 | Epoch 004 | Loss: 0.6575 | Test Acc: 0.5471
Seed 3 | Epoch 005 | Loss: 0.6521 | Test Acc: 0.5471
Seed 3 | Epoch 006 | Loss: 0.6466 | Test Acc: 0.5471
Seed 3 | Epoch 007 | Loss: 0.6401 | Test Acc: 0.5516
Seed 3 | Epoch 008 | Loss: 0.6325 | Test Acc: 0.5919
Seed 3 | Epoch 009 | Loss: 0.6259 | Test Acc: 0.6502
Seed 3 | Epoch 010 | Loss: 0.6173 | Test Acc: 0.6323
Seed 3 | Epoch 011 | Loss: 0.6072 | Test Acc: 0.6278
Seed 3 | Epoch 012 | Loss: 0.6048 | Test Acc: 0.6682
Seed 3 | Epoch 013 | Loss: 0.5974 | Test Acc: 0.6547
Seed 3 | Epoch 014 | Loss: 0.5865 | Test Acc: 0.6682
Seed 3 | Epoch 015 | Loss: 0.5829 | Test Acc: 0.6637
Seed 3 | Epoch 016 | Loss: 0.5746 | Test Acc: 0.6726
Seed 3 | Epoch 017 | Loss: 0.5682 | Test Acc: 0.6816
Seed 3 | Epoch 018 | Loss: 0.5623 | Test Acc: 0.6816
Seed 3 | Epoch 019 | Loss: 0.5602 | Test Acc: 

...

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▇▆▆▆▅▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▂▁▁
test_acc,▁▁▁▃▂▃▃▃▄▅▅▅▆▅▅▅▅▆▅▅▆▆▇▆▇▅▆▆▆▇▆██▇█▅█▇▇▇
train_acc,▁▁▁▂▃▃▃▃▅▅▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▆▇▇▇▇▇▇█▇▇▆▇███
epoch,50
loss,0.5415
test_acc,0.78027
train_acc,0.74719


Seed 29 | Epoch 001 | Loss: 0.6768 | Test Acc: 0.5964
Seed 29 | Epoch 002 | Loss: 0.6722 | Test Acc: 0.5964
Seed 29 | Epoch 003 | Loss: 0.6698 | Test Acc: 0.6188
Seed 29 | Epoch 004 | Loss: 0.6632 | Test Acc: 0.6413
Seed 29 | Epoch 005 | Loss: 0.6585 | Test Acc: 0.6413
Seed 29 | Epoch 006 | Loss: 0.6524 | Test Acc: 0.6413
Seed 29 | Epoch 007 | Loss: 0.6447 | Test Acc: 0.6368
Seed 29 | Epoch 008 | Loss: 0.6377 | Test Acc: 0.6682
Seed 29 | Epoch 009 | Loss: 0.6295 | Test Acc: 0.6592
Seed 29 | Epoch 010 | Loss: 0.6231 | Test Acc: 0.6547
Seed 29 | Epoch 011 | Loss: 0.6169 | Test Acc: 0.6726
Seed 29 | Epoch 012 | Loss: 0.6107 | Test Acc: 0.6502
Seed 29 | Epoch 013 | Loss: 0.6111 | Test Acc: 0.6413
Seed 29 | Epoch 014 | Loss: 0.6073 | Test Acc: 0.6771
Seed 29 | Epoch 015 | Loss: 0.5975 | Test Acc: 0.7040
Seed 29 | Epoch 016 | Loss: 0.5925 | Test Acc: 0.6502
Seed 29 | Epoch 017 | Loss: 0.5947 | Test Acc: 0.6457
Seed 29 | Epoch 018 | Loss: 0.5861 | Test Acc: 0.6592
Seed 29 | Epoch 019 | Loss: 

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▂▁▂▁▁▁▂
test_acc,▁▁▂▃▃▃▄▄▄▄▃▅▆▃▄▅▅▄▆▆▅▆▅▅▅▄▆▆▆▅▅▆▇▆▇▆▆▆█▇
train_acc,▁▁▂▂▂▃▄▄▅▅▅▅▅▆▆▆▆▆▇▆▆▇▇▇▇▇█▇▇█▇███▆██▇▇█
epoch,50
loss,0.5135
test_acc,0.72197
train_acc,0.76966


Seed 30 | Epoch 001 | Loss: 0.6841 | Test Acc: 0.6592
Seed 30 | Epoch 002 | Loss: 0.6739 | Test Acc: 0.6547
Seed 30 | Epoch 003 | Loss: 0.6711 | Test Acc: 0.6413
Seed 30 | Epoch 004 | Loss: 0.6650 | Test Acc: 0.6592
Seed 30 | Epoch 005 | Loss: 0.6603 | Test Acc: 0.6637
Seed 30 | Epoch 006 | Loss: 0.6530 | Test Acc: 0.6637
Seed 30 | Epoch 007 | Loss: 0.6441 | Test Acc: 0.6771
Seed 30 | Epoch 008 | Loss: 0.6367 | Test Acc: 0.6771
Seed 30 | Epoch 009 | Loss: 0.6251 | Test Acc: 0.6771
Seed 30 | Epoch 010 | Loss: 0.6164 | Test Acc: 0.7175
Seed 30 | Epoch 011 | Loss: 0.6054 | Test Acc: 0.6996
Seed 30 | Epoch 012 | Loss: 0.6024 | Test Acc: 0.6951
Seed 30 | Epoch 013 | Loss: 0.5896 | Test Acc: 0.6951
Seed 30 | Epoch 014 | Loss: 0.5851 | Test Acc: 0.6771
Seed 30 | Epoch 015 | Loss: 0.5772 | Test Acc: 0.6861
Seed 30 | Epoch 016 | Loss: 0.5712 | Test Acc: 0.7040
Seed 30 | Epoch 017 | Loss: 0.5643 | Test Acc: 0.7130
Seed 30 | Epoch 018 | Loss: 0.5622 | Test Acc: 0.7130
Seed 30 | Epoch 019 | Loss: 

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▇▇▇▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▁▂▁▁
test_acc,▂▂▁▂▂▃▃▃▅▄▃▄▅▅▅▆▅▆▆▆▇▅▇▆▇▇▇█▅▆▆█▇▇▇▆▆▆▇▇
train_acc,▁▁▃▃▂▅▄▅▆▆▆▇▇▇▇▇█▇████▇██████▇███████▇██
epoch,50
loss,0.50602
test_acc,0.73094
train_acc,0.77079
